In [21]:
import numpy as np
import gym

In [7]:
# Import and initialize Mountain Car Environment
env = gym.make('MountainCar-v0')
env.reset()

array([-0.41509222,  0.        ])

In [27]:
# Define Q-learning function
def QLearning(env, learning, discount, epsilon, min_eps, episodes):
    # Determine size of discretized state space
    num_states = (env.observation_space.high - env.observation_space.low)*\
                    np.array([10, 100])
    num_states = np.round(num_states, 0).astype(int) + 1
    
    # Initialize Q table
    Q = np.random.uniform(low = -1, high = 1, 
                          size = (num_states[0], num_states[1], 
                                  env.action_space.n))
    
    # Initialize variables to track rewards
    reward_list = []
    ave_reward_list = []
    
    # Calculate episodic reduction in epsilon
    reduction = (epsilon - min_eps)/episodes
    
    # Run Q learning algorithm
    for i in range(episodes):
        # Initialize parameters
        done = False
        tot_reward, reward = 0,0
        state = env.reset()
        
        # Discretize state
        state_adj = (state - env.observation_space.low)*np.array([10, 100])
        state_adj = np.round(state_adj, 0).astype(int)
    
        while done != True:   
            # Render environment for last five episodes
            if i >= (episodes - 20):
                env.render()
                
            # Determine next action - epsilon greedy strategy
            if np.random.random() < 1 - epsilon:
                action = np.argmax(Q[state_adj[0], state_adj[1]]) 
            else:
                action = np.random.randint(0, env.action_space.n)
                
            # Get next state and reward
            state2, reward, done, info = env.step(action) 
            
            # Discretize state2
            state2_adj = (state2 - env.observation_space.low)*np.array([10, 100])
            state2_adj = np.round(state2_adj, 0).astype(int)
            
            #Allow for terminal states
            if done and state2[0] >= 0.5:
                Q[state_adj[0], state_adj[1], action] = reward
                
            # Adjust Q value for current state
            else:
                delta = learning*(reward + 
                                 discount*np.max(Q[state2_adj[0], 
                                                   state2_adj[1]]) - 
                                 Q[state_adj[0], state_adj[1],action])
                Q[state_adj[0], state_adj[1],action] += delta
                                     
            # Update variables
            tot_reward += reward
            state_adj = state2_adj
        
        # Decay epsilon
        if epsilon > min_eps:
            epsilon -= reduction
        
        # Track rewards
        reward_list.append(tot_reward)
        
        if (i+1) % 100 == 0:
            ave_reward = np.mean(reward_list)
            ave_reward_list.append(ave_reward)
            reward_list = []
            
        if (i+1) % 100 == 0:    
            print('Episode {} Average Reward: {}'.format(i+1, ave_reward))
            
    env.close()
    
    return ave_reward_list

In [28]:
# Run Q-learning algorithm
rewards = QLearning(env, 0.1, 0.99, 0.1, 0, 5000)

[19 15]
Episode 100 Average Reward: -200.0
Episode 200 Average Reward: -200.0
Episode 300 Average Reward: -200.0
Episode 400 Average Reward: -200.0
Episode 500 Average Reward: -200.0
Episode 600 Average Reward: -199.17
Episode 700 Average Reward: -186.44
Episode 800 Average Reward: -193.46
Episode 900 Average Reward: -199.14
Episode 1000 Average Reward: -197.5
Episode 1100 Average Reward: -197.33
Episode 1200 Average Reward: -189.0
Episode 1300 Average Reward: -194.16
Episode 1400 Average Reward: -189.59
Episode 1500 Average Reward: -184.73
Episode 1600 Average Reward: -186.44
Episode 1700 Average Reward: -195.38
Episode 1800 Average Reward: -193.29
Episode 1900 Average Reward: -187.83
Episode 2000 Average Reward: -193.45
Episode 2100 Average Reward: -178.15
Episode 2200 Average Reward: -166.01
Episode 2300 Average Reward: -165.71
Episode 2400 Average Reward: -179.48
Episode 2500 Average Reward: -175.11
Episode 2600 Average Reward: -186.79
Episode 2700 Average Reward: -178.17
Episode 2

In [13]:
# Plot Rewards
plt.plot(100*(np.arange(len(rewards)) + 1), rewards)
plt.xlabel('Episodes')
plt.ylabel('Average Reward')
plt.title('Average Reward vs Episodes')
plt.show()

AttributeError: module 'matplotlib' has no attribute 'plot'

In [2]:
#